# Fine Tune the Minimal-Edit LLM

## Imports

Import all relevant packages

In [1]:
from prompts import minimal_prompt as prompt
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
    Trainer,
    DataCollatorForSeq2Seq,
)
from datasets import load_dataset, load_from_disk
import torch
import bitsandbytes as bnb
from os import path
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model
)

/home/jovyan/da231x/venv/lib/python3.10/site-packages/transformers/utils/hub.py:106: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## Model Setup

Load the model and tokenizer, then move the model onto the GPU.

Throw an error message if GPU is not accessible.

In [2]:
base_model_name = "LumiOpen/Viking-7B"
device = "cuda"
if not torch.cuda.is_available():
    raise RuntimeError("GPU is not available for training!")

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name, quantization_config=nf4_config
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
base_dataset_dir = "datasets"
minimal_dataset_path = path.join(base_dataset_dir, "minimal", "")
minimal_dataset = load_from_disk(minimal_dataset_path)
print(minimal_dataset)

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 402
    })
    validation: Dataset({
        features: ['source', 'target'],
        num_rows: 50
    })
    test: Dataset({
        features: ['source', 'target'],
        num_rows: 50
    })
})


## Process Input and Output

Load the input and output into a dictionary with the following structure:

- "input": [PROMPT] + input_text
- "output": output_text


In [4]:
def preprocess_function(examples):
    inputs = [prompt + example for example in examples["source"]]
    targets = [example for example in examples["target"]]
    return tokenizer(inputs, text_target=targets, padding=True)

In [5]:
tokenized_minimal_dataset = minimal_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [7]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    #target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

training_arguments = TrainingArguments(
    output_dir="tmp_model",
    num_train_epochs=3,
    optim="adamw_bnb_8bit",
    learning_rate=5e-5,
    bf16=True,
    logging_steps=1,
    per_device_train_batch_size=128,
)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_minimal_dataset["train"],
    eval_dataset=tokenized_minimal_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipykernel_12662/1813116116.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [8]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/jovyan/da231x/venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


RuntimeError: NVML_SUCCESS == r INTERNAL ASSERT FAILED at "/pytorch/c10/cuda/CUDACachingAllocator.cpp":1015, please report a bug to PyTorch. 